In [2]:
import pandas as pd
from datetime import date, timedelta
import urllib.request
import shutil
import os

In [8]:
# Batch download MTA turnstile files
start_date = date(2020, 1, 4)
end_date = date(2020, 4, 18)
while start_date <= end_date:
    filename = 'turnstile_'+start_date.strftime('%y%m%d')
    url = 'http://web.mta.info/developers/data/nyct/turnstile/'+filename+'.txt'
    print('Downloading '+filename+'.csv')
    with urllib.request.urlopen(url) as response, open('./data/'+filename+'.csv', 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
    start_date += timedelta(weeks=1)

In [7]:
# Import station data with latitude and longitude
pd.options.display.float_format = '{:.6f}'.format
geocoded = pd.read_csv('geocoded.csv')

In [47]:
def filter_mta(data):
    # Filter by regular records, remove PATH
    data_filtered = data[(data['DESC']=='REGULAR') | (data['DESC']=='RECOVR AUD')]
    data_filtered = data_filtered[data_filtered['DIVISION']!='PTH']

    # Get diff for each row
    data_filtered['ENTRIES_DIFF'] = data_filtered.groupby(['UNIT', 'SCP'])['ENTRIES'].diff()
    data_filtered['EXITS_DIFF'] = data_filtered.groupby(['UNIT', 'SCP'])['EXITS'].diff()
    data_filtered = data_filtered[(data_filtered['ENTRIES_DIFF']>=0) & (data_filtered['EXITS_DIFF']>=0)]

    # Remove some columns
    data_filtered.drop(columns=['STATION', 'LINENAME', 'DIVISION', 'DESC'], inplace=True)
    
    return data_filtered

In [48]:
# Batch process
directory = os.fsencode('./data')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print('Cleaning '+filename)

    # Import MTA weekly turnstile data
    data = pd.read_csv('./data/'+filename, parse_dates=['DATE'])
    data.columns = data.columns.str.strip()
    
    data_filtered = filter_mta(data)
    data_geocoded = pd.merge(data_filtered, geocoded, how='left', left_on=['UNIT'], right_on=['UNIT'])
    
    # Group and sum 
    data_cleaned = data_geocoded.groupby(['DATE', 'STATION', 'LATITUDE', 'LONGITUDE']).agg(
        {'ENTRIES_DIFF': 'sum', 'EXITS_DIFF': 'sum', 'LINENAME': lambda x: x.iloc[0]})
    data_cleaned = data_cleaned.astype({'ENTRIES_DIFF': 'int', 'EXITS_DIFF': 'int'})
    
    # Export
    data_cleaned.to_csv('./cleaned/cleaned_'+filename, float_format='%.6f')

Cleaning turnstile_200104.csv
Cleaning turnstile_200111.csv
Cleaning turnstile_200118.csv
Cleaning turnstile_200125.csv
Cleaning turnstile_200201.csv
Cleaning turnstile_200208.csv
Cleaning turnstile_200215.csv
Cleaning turnstile_200222.csv
Cleaning turnstile_200229.csv
Cleaning turnstile_200307.csv
Cleaning turnstile_200314.csv
Cleaning turnstile_200321.csv
Cleaning turnstile_200328.csv
Cleaning turnstile_200404.csv
Cleaning turnstile_200411.csv
Cleaning turnstile_200418.csv
Cleaning turnstile_200425.csv


In [46]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(data_cleaned)

ENTRIES_DIFF  EXITS_DIFF  \
DATE       STATION         LATITUDE  LONGITUDE                              
2019-12-28 1 AVE           40.730901 -73.981719         10904       12838   
           103 ST          40.790582 -73.947473          5549        5681   
                           40.796105 -73.961399          2415        1451   
                           40.799354 -73.968329          6515        3300   
           103 ST-CORONA   40.749858 -73.862672         14003        9855   
           104 ST          40.695184 -73.844326          1310         641   
           110 ST          40.795066 -73.944297         12002       11726   
           110 ST-CATHEDRL 40.804032 -73.966742         12382        6234   
           110 ST-CPN      40.799110 -73.951807             0         281   
           111 ST          40.697405 -73.836354          1157         604   
                           40.751760 -73.855183          8274        4944   
           116 ST          40.798574 -73.941593         16130       10224   
                           40.802098 -73.949625             8          62   
                           40.805072 -73.954833          9140        5296   
           116 ST-COLUMBIA 40.808190 -73.964124          3643        1933   
           121 ST          40.700536 -73.828382          1172         749   
           125 ST          40.804406 -73.937452         14239       16242   
                           40.808076 -73.945906           108         136   
                           40.811056 -73.952386         38714       36002   
                           40.815596 -73.958395          4336        4856   
           135 ST          40.814459 -73.940992            82          82   
                           40.817902 -73.947644         11106        5691   
           137 ST-CITY COL 40.821994 -73.953674         12584        9870   
           138 ST-3 AVE    40.810512 -73.926165          3212        1748   
           138 ST-GR CONC  40.813208 -73.929877          1191        1275   
           14 ST           40.738007 -73.998381         20260       13398   
                           40.740388 -74.002104         23943       24325   
           14 ST-6 AVE     40.737348 -73.996900         40504       57192   
           14 ST-UNION SQ  40.734836 -73.990688        270408      313380   
           145 ST          40.820402 -73.936315             0           2   
                           40.824787 -73.944232         28538       26140   
                           40.826426 -73.950412         12486        9648   
           148 ST-LENOX    40.823877 -73.936443             1          11   
           149 ST-3 AVE    40.816132 -73.917754         21026        8864   
           149 ST-GR CONC  40.818429 -73.926927          9330       15312   
           15 ST-PROSPECT  40.660376 -73.979509          5854        4080   
           155 ST          40.829934 -73.938632          2145        1182   
                           40.830551 -73.941486          1566        1127   
           157 ST          40.833879 -73.944726         12556        8082   
           161 ST-YANKEE   40.827888 -73.925736         71832       40206   
           163 ST-AMSTERDM 40.835957 -73.939898          2453        2001   
           167 ST          40.833773 -73.918430          9302        3154   
                           40.835535 -73.921380          4913        5688   
           168 ST-BROADWAY 40.840778 -73.940091         30843       18165   
           169 ST          40.710459 -73.793600          8588        6488   
           170 ST          40.839301 -73.913355          7848        4188   
                           40.840048 -73.917775          4535        3189   
           174 ST          40.837382 -73.887659          5608        4804   
           174-175 ST      40.845892 -73.910136          6208        3242   
           175 ST          40.847369 -73.939683         16346        8432   
           176 ST          40.848635 -73.9124